## Empty server

In [1]:
from nanover.omni import OmniRunner

runner = OmniRunner.with_basic_server()
runner.print_basic_info()

Serving "ragzo: NanoVer iMD Server" (ws://localhost:38801), discoverable on all interfaces on port 54545
Available simulations:



## Client connected to empty server

In [4]:
from nanover.websocket import NanoverImdClient

client = NanoverImdClient.from_runner(runner)

## Simulation

In [5]:
from nanover.openmm import OpenMMSimulation

input_xml = "openmm_files/nanotube.xml"
nanotube_simulation = OpenMMSimulation.from_xml_path(input_xml)
nanotube_simulation.load()

## Publish simulation frames via client

In [ ]:
# NBVAL_SKIP
from nanover.omni import OmniRunner

fake_runner = OmniRunner(client)
fake_runner.add_simulation(nanotube_simulation)

fake_runner.load(0)